<a href="https://colab.research.google.com/github/hosseinmiripy/bottelegram/blob/main/%D8%A7%D8%B3%D9%85%D8%A7%D8%B1%D8%AA_%D9%85%D8%A7%D9%86%DB%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
🐋 Smart Money Whale Hunter - Professional Edition v2.1 (ENHANCED)
==================================================================
✅ اضافه شدن USDT Dominance
✅ اضافه شدن Market Phase Detection
✅ اضافه شدن حداقل تفاوت امتیاز (25)
✅ بهبود Stop Loss Validation
✅ محدودیت ریسک به 3%
✅ اعتبارسنجی نهایی سیگنال
✅ Position Size Recommendation
"""

# ==================== بخش 1: نصب کتابخانه‌ها ====================
import sys
import subprocess

def install_requirements():
    """نصب هوشمند کتابخانه‌ها با رفع مشکل Kaleido"""
    packages = {
        'core': ['pandas', 'numpy', 'scipy', 'scikit-learn'],
        'data': ['ccxt', 'requests'],
        'analysis': ['ta'],
        'visualization': ['matplotlib', 'seaborn', 'mplfinance', 'plotly'],
        'system': ['psutil', 'python-dateutil'],
    }

    print("📦 نصب کتابخانه‌های اصلی...")
    for category, libs in packages.items():
        for lib in libs:
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', lib],
                          capture_output=True)

    # نصب Kaleido با مدیریت خطا
    print("🎨 نصب Kaleido...")
    try:
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'kaleido==0.2.1'],
                      check=True, capture_output=True)
        print("✅ Kaleido نصب شد")
    except:
        print("⚠️ Kaleido نصب نشد - از matplotlib استفاده می‌شود")

    print("🔧 نصب تلگرام...")
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'python-telegram-bot'],
                   capture_output=True)
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'telegram'],
                   capture_output=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'python-telegram-bot==20.8'],
                   check=True)

    print("✅ نصب کامل شد\n")


try:
    install_requirements()
except Exception as e:
    print(f"⚠️ خطا در نصب: {e}")

# ==================== بخش 2: Import ====================
import os
import time
import json
import logging
import warnings
import hashlib
import pickle
import gc
import traceback
import math
import re
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field
from collections import defaultdict, deque
from threading import Lock
from io import BytesIO
import tempfile

import pandas as pd
import numpy as np
import ccxt
import ta
import requests
import psutil

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
from matplotlib.patches import Rectangle, FancyBboxPatch, Polygon
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import nest_asyncio
nest_asyncio.apply()

TELEGRAM_AVAILABLE = False
try:
    from telegram import Update, InputFile
    from telegram.ext import Application, CommandHandler, ContextTypes
    TELEGRAM_AVAILABLE = True
    print("✅ تلگرام بارگذاری شد")
except ImportError as e:
    print(f"⚠️ تلگرام غیرفعال: {e}")

warnings.filterwarnings("ignore")

# ==================== بخش 3: تنظیمات محیط ====================
COLAB_ENV = False
try:
    import google.colab
    COLAB_ENV = True
    from google.colab import drive

    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    BASE_DIR = '/content/drive/MyDrive/smart_money_whale_hunter'
    print("🌟 محیط Google Colab")
except:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
    print("🖥️ محیط محلی")

for dir_name in ['logs', 'charts', 'cache/ohlcv', 'cache/disk', 'temp']:
    os.makedirs(f"{BASE_DIR}/{dir_name}", exist_ok=True)

print(f"📂 BASE_DIR: {BASE_DIR}\n")

# ==================== بخش 4: لاگینگ ====================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f"{BASE_DIR}/logs/whale_hunter.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# ==================== بخش 5: تنظیمات کلی ====================
TELEGRAM_BOT_TOKEN = "7307185304:AAF9zG3vM0Pc74WkiIDVEnRxu41OJYdyql8"
TELEGRAM_CHANNEL_ID = "@finance_hossein"

ADMIN_USERS = ["@hosseinmiri3", 1150906790]  # یوزرنیم و یوزر آیدی ادمین

BOT_SETTINGS = {
    'max_memory_cache_size': 300,
    'cache_max_age_hours': 2,
    'cleanup_interval_hours': 1,
    'min_score_difference': 25,  # ⬅️ جدید
    'max_risk_percent': 3.0,     # ⬅️ جدید
    'sl_buffer_percent': 0.5,    # ⬅️ جدید
}

EXCHANGES_CONFIG = {
    'binance': {
        'class': ccxt.binance,
        'max_requests_per_minute': 50,
        'weight': 0.5,  # کاهش وزن binance
        'cooldown': 30,
    },
    'kucoin': {
        'class': ccxt.kucoin,
        'max_requests_per_minute': 15,
        'weight': 1.0,  # افزایش اولویت kucoin
        'cooldown': 60,
    },
    'gate': {
        'class': ccxt.gate,
        'max_requests_per_minute': 20,
        'weight': 0.9,  # افزایش اولویت gate
        'cooldown': 50,
    },
    'bybit': {  # صرافی جدید
        'class': ccxt.bybit,
        'max_requests_per_minute': 30,
        'weight': 0.8,
        'cooldown': 45,
    },
}

exchange_state = {}
exchange_data = {}
for ex_id in EXCHANGES_CONFIG:
    state = {
        'request_count': 0,
        'last_reset_time': time.time(),
        'in_cooldown': False,
        'cooldown_until': 0,
        'success_count': 0,
        'fail_count': 0,
    }
    exchange_state[ex_id] = state
    exchange_data[ex_id] = state

request_lock = Lock()
request_count_lock = request_lock

memory_cache = {}
memory_cache_time = {}
memory_cache_access_time = {}
CACHE_MAX_SIZE = 300
CACHE_MAX_AGE_HOURS = 2
last_cleanup = time.time()

# ==================== بخش 6: کلاس‌های داده ====================
@dataclass
class OrderBlock:
    high: float
    low: float
    time: int
    bias: str
    strength: float
    touched: bool = False
    volume: float = 0

@dataclass
class FairValueGap:
    top: float
    bottom: float
    time: int
    bias: str
    filled: bool = False
    size: float = 0

@dataclass
class MarketStructure:
    type: str
    direction: str
    price: float
    time: int
    strength: float
    confirmed: bool = True

@dataclass
class WhaleActivity:
    timestamp: int
    volume: float
    price: float
    type: str
    strength: float
    impact: float = 0

@dataclass
class LiquidityZone:
    price: float
    type: str
    strength: float
    time: int

@dataclass
class TradingSignal:
    """سیگنال معاملاتی - نسخه بهبود یافته"""
    symbol: str
    direction: str
    entry_price: float
    stop_loss: float
    targets: List[float]
    confidence: float
    timeframe: str
    reasons: List[str]
    market_context: Dict
    timestamp: datetime
    risk_reward: float = 0
    chart_path: Optional[str] = None
    position_size_recommendation: str = ""  # ⬅️ جدید
    risk_percent: float = 0.0  # ⬅️ جدید

    def __post_init__(self):
        # محاسبه R:R
        if self.direction == 'LONG':
            self.risk_reward = (self.targets[0] - self.entry_price) / (self.entry_price - self.stop_loss)
        else:
            self.risk_reward = (self.entry_price - self.targets[0]) / (self.stop_loss - self.entry_price)

        # محاسبه درصد ریسک
        self.risk_percent = abs(self.stop_loss - self.entry_price) / self.entry_price * 100

        # توصیه حجم معامله
        self.position_size_recommendation = (
            f"با ریسک 1% سرمایه: حجم = (سرمایه × 0.01) / (فاصله تا SL)\n"
            f"Risk per trade: {self.risk_percent:.2f}% | "
            f"Suggested: Use 1% of capital"
        )

# ==================== بخش 7: توابع کمکی ====================

def get_cache_key(*args):
    return "_".join(str(arg) for arg in args)

def clean_memory_cache(max_items=None, force_cleanup=False):
    global memory_cache, memory_cache_access_time

    if max_items is None:
        max_items = BOT_SETTINGS.get('max_memory_cache_size', 300) // 2

    if len(memory_cache) > max_items or force_cleanup:
        sorted_keys = sorted(
            memory_cache_access_time.items(),
            key=lambda x: x[1]
        )

        keys_to_remove = [k for k, _ in sorted_keys[:len(sorted_keys) - max_items]]

        for key in keys_to_remove:
            if key in memory_cache:
                del memory_cache[key]
            if key in memory_cache_access_time:
                del memory_cache_access_time[key]

        logger.info(f"🧹 پاکسازی کش حافظه: {len(keys_to_remove)} آیتم حذف شد")
        gc.collect()

def validate_data(df):
    if df is None or len(df) < 20:
        return False

    try:
        null_count = df.isnull().sum().sum()
        if null_count > 0:
            if null_count < len(df) * 0.03:
                df.ffill(inplace=True)
                df.bfill(inplace=True)
                if df.isnull().sum().sum() > 0:
                    return False
            else:
                return False

        price_changes = df['close'].pct_change().abs()
        if price_changes.max() > 0.5:
            extreme_changes = price_changes[price_changes > 0.25]
            if len(extreme_changes) > len(df) * 0.05:
                return False

        zero_volume_ratio = (df['volume'] == 0).sum() / len(df)
        if zero_volume_ratio > 0.15:
            return False

        return True
    except Exception as e:
        logger.error(f"خطا در اعتبارسنجی داده: {e}")
        return False

def fetch_data(symbol, timeframe, limit=500, force_fresh=False, max_age_hours=1,
               max_retries=5, delay=2, incremental=False):
    cache_key = get_cache_key("fetch_data", symbol, timeframe, limit)

    if not force_fresh and cache_key in memory_cache:
        cache_age = time.time() - memory_cache_access_time.get(cache_key, 0)
        if cache_age < max_age_hours * 3600:
            memory_cache_access_time[cache_key] = time.time()
            logger.info(f"✅ کش حافظه: {symbol}/{timeframe} (سن: {cache_age/60:.1f} دقیقه)")
            return memory_cache[cache_key].copy()

    ex_id = None

    for attempt in range(max_retries):
        try:
            ex_id = get_best_exchange(symbol, "ohlcv")

            if ex_id not in EXCHANGES_CONFIG:
                logger.warning(f"صرافی {ex_id} در پیکربندی نیست")
                continue

            exchange_class = EXCHANGES_CONFIG[ex_id]['class']
            exchange = exchange_class({
                'enableRateLimit': True,
                'timeout': 30000,
                'options': {'defaultType': 'spot'}
            })

            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

            if not ohlcv or len(ohlcv) < 20:
                logger.warning(f"داده ناکافی از {ex_id}: {len(ohlcv) if ohlcv else 0} کندل")
                continue

            df = pd.DataFrame(
                ohlcv,
                columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']
            )
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)

            if not validate_data(df):
                logger.warning(f"داده نامعتبر از {ex_id}")
                continue

            memory_cache[cache_key] = df
            memory_cache_access_time[cache_key] = time.time()

            if len(memory_cache) > BOT_SETTINGS.get('max_memory_cache_size', 300):
                clean_memory_cache()

            with request_count_lock:
                exchange_data[ex_id]['success_count'] += 1

            logger.info(f"✅ دریافت {symbol}/{timeframe}: {len(df)} کندل از {ex_id}")
            return df.copy()

        except ccxt.RateLimitExceeded:
            ex_name = ex_id if ex_id else "نامشخص"
            logger.warning(f"⏳ محدودیت نرخ {ex_name} - تلاش {attempt+1}/{max_retries}")
            if ex_id:
                handle_rate_limit(ex_id)
            if attempt < max_retries - 1:
                time.sleep(delay * (attempt + 1))

        except ccxt.ExchangeError as e:
            ex_name = ex_id if ex_id else "نامشخص"
            logger.error(f"❌ خطای صرافی {ex_name}: {str(e)[:100]}")
            if ex_id:
                with request_count_lock:
                    exchange_data[ex_id]['fail_count'] += 1
            if attempt < max_retries - 1:
                time.sleep(delay)

        except Exception as e:
            ex_name = ex_id if ex_id else "نامشخص"
            logger.error(f"❌ خطای غیرمنتظره {ex_name}: {str(e)[:100]}")
            logger.error(traceback.format_exc())
            if attempt < max_retries - 1:
                time.sleep(delay)

    logger.error(f"❌ دریافت {symbol}/{timeframe} ناموفق بود بعد از {max_retries} تلاش")
    return None

def save_to_disk_cache(key: str, data: Any):
    try:
        hash_key = hashlib.md5(key.encode()).hexdigest()
        path = f"{BASE_DIR}/cache/disk/{hash_key}.pkl"

        with open(path, 'wb') as f:
            pickle.dump(data, f, protocol=4)
        return True
    except Exception as e:
        logger.error(f"خطا ذخیره دیسک: {e}")
        return False

def load_from_disk_cache(key: str) -> Optional[Any]:
    try:
        hash_key = hashlib.md5(key.encode()).hexdigest()
        path = f"{BASE_DIR}/cache/disk/{hash_key}.pkl"

        if not os.path.exists(path):
            return None

        if time.time() - os.path.getmtime(path) > CACHE_MAX_AGE_HOURS * 3600:
            os.remove(path)
            return None

        with open(path, 'rb') as f:
            return pickle.load(f)
    except:
        return None

def cleanup_files():
    global last_cleanup

    if time.time() - last_cleanup < 3600:
        return

    logger.info("🗑️ پاکسازی فایل‌ها...")

    charts_dir = f"{BASE_DIR}/charts"
    if os.path.exists(charts_dir):
        files = sorted(
            [(f, os.path.getmtime(os.path.join(charts_dir, f)))
             for f in os.listdir(charts_dir) if f.endswith('.png')],
            key=lambda x: x[1], reverse=True
        )
        for f, _ in files[20:]:
            try:
                os.remove(os.path.join(charts_dir, f))
            except:
                pass

    for cache_dir in [f"{BASE_DIR}/cache/ohlcv", f"{BASE_DIR}/cache/disk"]:
        if os.path.exists(cache_dir):
            for f in os.listdir(cache_dir):
                path = os.path.join(cache_dir, f)
                try:
                    if time.time() - os.path.getmtime(path) > 12 * 3600:
                        os.remove(path)
                except:
                    pass

    last_cleanup = time.time()
    logger.info("✅ فایل‌ها پاکسازی شد")

cleanup_drive_files = cleanup_files

def monitor_resources() -> Dict:
    try:
        memory = psutil.virtual_memory()

        status = {
            'memory_percent': memory.percent,
            'memory_available_gb': memory.available / (1024**3),
            'cache_items': len(memory_cache),
        }

        if memory.percent > 80:
            logger.warning(f"🚨 حافظه بالا: {memory.percent:.1f}%")
            clean_memory_cache(force_cleanup=True)
        elif memory.percent > 60:
            clean_memory_cache(force_cleanup=True)

        return status
    except:
        return {}

def get_best_exchange(symbol: str, request_type: str = "ohlcv") -> str:
    with request_count_lock:
        now = time.time()
        available = []

        for ex_id, config in EXCHANGES_CONFIG.items():
            state = exchange_data[ex_id]

            if state['in_cooldown']:
                if now > state['cooldown_until']:
                    state['in_cooldown'] = False
                    logger.info(f"✅ {ex_id} از cooldown خارج شد")
                else:
                    continue

            if now - state['last_reset_time'] > 60:
                state['request_count'] = 0
                state['last_reset_time'] = now

            max_requests = config['max_requests_per_minute']
            if state['request_count'] < max_requests * 0.8:
                score = config['weight']

                total_attempts = state['success_count'] + state['fail_count']
                if total_attempts > 0:
                    success_rate = state['success_count'] / total_attempts
                    score *= success_rate

                available.append((ex_id, score))

        if not available:
            logger.warning("⚠️ هیچ صرافی در دسترس نیست، انتظار برای cooldown...")

            min_cooldown_ex = min(
                EXCHANGES_CONFIG.keys(),
                key=lambda x: exchange_data[x]['cooldown_until']
            )

            wait_time = max(1, exchange_data[min_cooldown_ex]['cooldown_until'] - now)
            logger.info(f"⏳ انتظار {wait_time:.1f} ثانیه برای {min_cooldown_ex}...")
            time.sleep(wait_time)

            return get_best_exchange(symbol, request_type)

        best_ex = max(available, key=lambda x: x[1])[0]
        exchange_data[best_ex]['request_count'] += 1

        logger.info(f"📊 انتخاب صرافی: {best_ex} (امتیاز: {dict(available)[best_ex]:.2f})")
        return best_ex

def handle_rate_limit(ex_id: str):
    with request_lock:
        state = exchange_state[ex_id]
        state['in_cooldown'] = True
        state['cooldown_until'] = time.time() + EXCHANGES_CONFIG[ex_id]['cooldown']
        logger.warning(f"⏸️ {ex_id} در کولداون")

# ==================== بخش 8: کلاس اصلی ====================
class SmartMoneyWhaleHunter:

    def __init__(self):
        logger.info("🚀 راه‌اندازی SmartMoneyWhaleHunter...")

        self.exchanges = self._init_exchanges()

        self.swing_length = 50
        self.internal_length = 10
        self.ob_atr_period = 14
        self.ob_atr_multiplier = 0.5
        self.fvg_threshold = 0.001

        self.whale_volume_threshold = 3.0
        self.whale_price_impact = 0.005

        self.liquidity_lookback = 100

        logger.info("✅ سیستم آماده")

    def _init_exchanges(self) -> Dict:
        exchanges = {}

        for ex_id, config in EXCHANGES_CONFIG.items():
            try:
                exchanges[ex_id] = config['class']({
                    'enableRateLimit': True,
                    'timeout': 30000,
                    'options': {'defaultType': 'spot'}
                })
                logger.info(f"✅ {ex_id}")
            except Exception as e:
                logger.error(f"❌ {ex_id}: {e}")

        return exchanges

    def fetch_ohlcv_with_cache(self, symbol: str, timeframe: str, limit: int = 500,
                                force_fresh: bool = False, max_age_hours: float = 1) -> Optional[pd.DataFrame]:
        try:
            logger.info(f"📥 درخواست داده برای {symbol} در تایم‌فریم {timeframe}...")

            df = fetch_data(
                symbol=symbol,
                timeframe=timeframe,
                limit=limit,
                force_fresh=force_fresh,
                max_age_hours=max_age_hours,
                max_retries=5,
                delay=2
            )

            if df is not None and len(df) > 0:
                if validate_data(df):
                    logger.info(f"✅ داده‌های معتبر {symbol} دریافت شد: {len(df)} کندل")
                    return df
                else:
                    logger.warning(f"⚠️ داده‌های {symbol} اعتبارسنجی نشد، تلاش مجدد...")
                    df = fetch_data(
                        symbol=symbol,
                        timeframe=timeframe,
                        limit=limit,
                        force_fresh=True,
                        max_age_hours=max_age_hours
                    )
                    if df is not None and validate_data(df):
                        logger.info(f"✅ داده‌های معتبر در تلاش دوم دریافت شد")
                        return df
                    else:
                        logger.error(f"❌ داده‌های معتبر برای {symbol} دریافت نشد")
                        return None
            else:
                logger.warning(f"⚠️ داده‌ای برای {symbol} در {timeframe} دریافت نشد")
                return None

        except Exception as e:
            logger.error(f"❌ خطا در دریافت داده {symbol}/{timeframe}: {e}")
            logger.error(traceback.format_exc())
            return None

    def fetch_ohlcv(self, symbol: str, timeframe: str, limit: int = 500) -> Optional[pd.DataFrame]:
        return self.fetch_ohlcv_with_cache(symbol, timeframe, limit, force_fresh=False)


    def fetch_market_dominance(self) -> Dict:
        """دریافت داده‌های بازار با USDT Dominance و Market Phase Detection"""
        cache_key = get_cache_key('dominance')

        # بررسی کش
        if cache_key in memory_cache:
            if time.time() - memory_cache_access_time.get(cache_key, 0) < 600:  # 10 دقیقه
                return memory_cache[cache_key]

        dominance = {
            'btc_dominance': 0,
            'btc_trend': 'neutral',
            'usdt_dominance': 0,
            'usdt_trend': 'neutral',
            'total_mcap': 0,
            'market_phase': 'neutral',
        }

        try:
            # دریافت BTC Dominance از CoinGecko
            resp = requests.get('https://api.coingecko.com/api/v3/global', timeout=10)

            if resp.status_code == 200:
                data = resp.json()['data']
                dominance['btc_dominance'] = data['market_cap_percentage'].get('btc', 0)
                dominance['total_mcap'] = data.get('total_market_cap', {}).get('usd', 0)
                logger.info(f"BTC Dominance: {dominance['btc_dominance']:.2f}%")

            # تحلیل روند BTC.D از قیمت BTC
            btc_d_data = self.fetch_ohlcv_with_cache('BTC/USDT', '1h', limit=50)
            if btc_d_data is not None:
                btc_d_data['sma'] = btc_d_data['close'].rolling(20).mean()
                if btc_d_data['close'].iloc[-1] > btc_d_data['sma'].iloc[-1]:
                    dominance['btc_trend'] = 'rising'
                else:
                    dominance['btc_trend'] = 'falling'

            # محاسبه USDT Dominance از Volume
            try:
                if 'binance' in self.exchanges:
                    exchange = self.exchanges['binance']
                    tickers = exchange.fetch_tickers()

                    usdt_volume = 0
                    total_volume = 0

                    for symbol, ticker in tickers.items():
                        quote_vol = ticker.get('quoteVolume', 0)
                        if quote_vol and quote_vol > 0:
                            total_volume += quote_vol
                            if 'USDT' in symbol:
                                usdt_volume += quote_vol

                    if total_volume > 0:
                        dominance['usdt_dominance'] = (usdt_volume / total_volume) * 100
                    else:
                        dominance['usdt_dominance'] = 5.0

                    # تعیین روند USDT
                    if dominance['usdt_dominance'] > 6:
                        dominance['usdt_trend'] = 'rising'  # Risk-Off - خروج پول
                        logger.info("USDT.D بالا - حالت Risk-Off")
                    else:
                        dominance['usdt_trend'] = 'falling'  # Risk-On - ورود پول

                    logger.info(f"USDT Dominance: {dominance['usdt_dominance']:.2f}%")
                else:
                    dominance['usdt_dominance'] = 5.0
            except Exception as e:
                logger.warning(f"خطا در محاسبه USDT.D: {e}")
                dominance['usdt_dominance'] = 5.0

            # تشخیص فاز بازار (Market Phase Detection)
            btc_d = dominance['btc_dominance']
            btc_trend = dominance['btc_trend']
            usdt_d = dominance['usdt_dominance']

            if btc_trend == 'rising' and btc_d < 45:
                dominance['market_phase'] = 'altseason'
                logger.info("🌟 فاز بازار: Altseason")

            elif btc_trend == 'falling' and usdt_d > 6:
                dominance['market_phase'] = 'bear_market'
                logger.info("🐻 فاز بازار: Bear Market")

            elif btc_d > 55:
                dominance['market_phase'] = 'btc_accumulation'
                logger.info("🟠 فاز بازار: BTC Accumulation")

            else:
                dominance['market_phase'] = 'neutral'
                logger.info("➖ فاز بازار: Neutral")

            # ذخیره در کش
            memory_cache[cache_key] = dominance
            memory_cache_access_time[cache_key] = time.time()

            logger.info(f"Market Context: BTC.D={btc_d:.2f}% ({btc_trend}), USDT.D={usdt_d:.2f}%, Phase={dominance['market_phase']}")

        except Exception as e:
            logger.error(f"خطا در دریافت Dominance: {e}")

        return dominance

    def detect_swing_points(self, df: pd.DataFrame, length: int) -> Tuple[pd.Series, pd.Series]:
        """تشخیص Swing Points"""
        highs = pd.Series(index=df.index, dtype=float)
        lows = pd.Series(index=df.index, dtype=float)

        for i in range(length, len(df) - length):
            if df['high'].iloc[i] == df['high'].iloc[i-length:i+length+1].max():
                highs.iloc[i] = df['high'].iloc[i]

            if df['low'].iloc[i] == df['low'].iloc[i-length:i+length+1].min():
                lows.iloc[i] = df['low'].iloc[i]

        return highs.dropna(), lows.dropna()

    def detect_market_structure(self, df: pd.DataFrame, tf: str) -> List[MarketStructure]:
        """تشخیص BOS & CHoCH"""
        structures = []

        length = self.swing_length if tf in ['1h', '4h'] else self.internal_length
        swing_highs, swing_lows = self.detect_swing_points(df, length)

        if len(swing_highs) < 2 or len(swing_lows) < 2:
            return structures

        all_swings = []
        for idx in swing_highs.index:
            all_swings.append(('high', idx, swing_highs[idx]))
        for idx in swing_lows.index:
            all_swings.append(('low', idx, swing_lows[idx]))

        all_swings.sort(key=lambda x: x[1])

        last_high = None
        last_low = None
        trend = None

        for swing_type, swing_time, swing_price in all_swings:
            if swing_type == 'high':
                if last_high is not None:
                    if swing_price > last_high:
                        structure_type = 'CHoCH' if trend == 'bearish' else 'BOS'

                        structures.append(MarketStructure(
                            type=structure_type,
                            direction='bullish',
                            price=last_high,
                            time=int(swing_time.timestamp() * 1000),
                            strength=abs(swing_price - last_high) / last_high * 100
                        ))

                        trend = 'bullish'

                last_high = swing_price

            else:  # low
                if last_low is not None:
                    if swing_price < last_low:
                        structure_type = 'CHoCH' if trend == 'bullish' else 'BOS'

                        structures.append(MarketStructure(
                            type=structure_type,
                            direction='bearish',
                            price=last_low,
                            time=int(swing_time.timestamp() * 1000),
                            strength=abs(swing_price - last_low) / last_low * 100
                        ))

                        trend = 'bearish'

                last_low = swing_price

        logger.info(f"Market Structure: {len(structures)} ساختار شناسایی شد")
        return structures

    def detect_order_blocks(self, df: pd.DataFrame, structures: List[MarketStructure]) -> List[OrderBlock]:
        """تشخیص Order Blocks با فیلتر ATR"""
        obs = []

        df['atr'] = ta.volatility.average_true_range(
            df['high'], df['low'], df['close'], window=self.ob_atr_period
        )

        for structure in structures[-10:]:
            st_time = pd.to_datetime(structure.time, unit='ms')

            pre_break = df[df.index < st_time].tail(20)
            if len(pre_break) == 0:
                continue

            if structure.direction == 'bullish':
                bearish = pre_break[pre_break['close'] < pre_break['open']]

                if len(bearish) > 0:
                    ob = bearish.iloc[-1]
                    candle_range = ob['high'] - ob['low']

                    if candle_range > df['atr'].iloc[-1] * self.ob_atr_multiplier:
                        vol_strength = (ob['volume'] / df['volume'].mean()) * 100
                        strength = min(100, (vol_strength + structure.strength) / 2)

                        obs.append(OrderBlock(
                            high=ob['high'],
                            low=ob['low'],
                            time=int(ob.name.timestamp() * 1000),
                            bias='bullish',
                            strength=strength,
                            volume=ob['volume']
                        ))

            else:  # bearish
                bullish = pre_break[pre_break['close'] > pre_break['open']]

                if len(bullish) > 0:
                    ob = bullish.iloc[-1]
                    candle_range = ob['high'] - ob['low']

                    if candle_range > df['atr'].iloc[-1] * self.ob_atr_multiplier:
                        vol_strength = (ob['volume'] / df['volume'].mean()) * 100
                        strength = min(100, (vol_strength + structure.strength) / 2)

                        obs.append(OrderBlock(
                            high=ob['high'],
                            low=ob['low'],
                            time=int(ob.name.timestamp() * 1000),
                            bias='bearish',
                            strength=strength,
                            volume=ob['volume']
                        ))

        obs.sort(key=lambda x: x.strength, reverse=True)

        logger.info(f"Order Blocks: {len(obs)} OB یافت شد")
        return obs[:5]

    def detect_fvg(self, df: pd.DataFrame) -> List[FairValueGap]:
        """تشخیص Fair Value Gaps"""
        fvgs = []

        for i in range(2, len(df)):
            c1 = df.iloc[i-2]
            c2 = df.iloc[i-1]
            c3 = df.iloc[i]

            # Bullish FVG
            if c3['low'] > c1['high']:
                gap = c3['low'] - c1['high']

                if gap / c1['high'] > self.fvg_threshold:
                    fvgs.append(FairValueGap(
                        top=c3['low'],
                        bottom=c1['high'],
                        time=int(c2.name.timestamp() * 1000),
                        bias='bullish',
                        size=gap
                    ))

            # Bearish FVG
            elif c3['high'] < c1['low']:
                gap = c1['low'] - c3['high']

                if gap / c1['low'] > self.fvg_threshold:
                    fvgs.append(FairValueGap(
                        top=c1['low'],
                        bottom=c3['high'],
                        time=int(c2.name.timestamp() * 1000),
                        bias='bearish',
                        size=gap
                    ))

        logger.info(f"FVG: {len(fvgs)} شکاف یافت شد")
        return fvgs[-10:]

    def calculate_zones(self, df: pd.DataFrame) -> Dict:
        """محاسبه Premium/Discount Zones"""
        swing_highs, swing_lows = self.detect_swing_points(df, self.swing_length)

        if len(swing_highs) == 0 or len(swing_lows) == 0:
            return {}

        high = swing_highs.iloc[-1]
        low = swing_lows.iloc[-1]
        diff = high - low

        zones = {
            'premium': {'top': high, 'bottom': high - diff * 0.382},
            'equilibrium': {'top': low + diff * 0.618, 'bottom': low + diff * 0.382},
            'discount': {'top': low + diff * 0.382, 'bottom': low},
            'range_high': high,
            'range_low': low,
        }

        current = df['close'].iloc[-1]

        if current >= zones['premium']['bottom']:
            zones['current_zone'] = 'premium'
        elif current <= zones['discount']['top']:
            zones['current_zone'] = 'discount'
        else:
            zones['current_zone'] = 'equilibrium'

        return zones

    def detect_liquidity_zones(self, df: pd.DataFrame) -> List[LiquidityZone]:
        """تشخیص نواحی نقدینگی"""
        zones = []

        swing_highs, swing_lows = self.detect_swing_points(df, 20)

        # Equal Highs
        highs_list = swing_highs.tolist()
        for i in range(len(highs_list) - 1):
            if abs(highs_list[i] - highs_list[i+1]) / highs_list[i] < 0.002:
                zones.append(LiquidityZone(
                    price=highs_list[i],
                    type='sell',
                    strength=70,
                    time=int(swing_highs.index[i].timestamp() * 1000)
                ))

        # Equal Lows
        lows_list = swing_lows.tolist()
        for i in range(len(lows_list) - 1):
            if abs(lows_list[i] - lows_list[i+1]) / lows_list[i] < 0.002:
                zones.append(LiquidityZone(
                    price=lows_list[i],
                    type='buy',
                    strength=70,
                    time=int(swing_lows.index[i].timestamp() * 1000)
                ))

        logger.info(f"Liquidity: {len(zones)} ناحیه نقدینگی")
        return zones

    def detect_whale_activity(self, df: pd.DataFrame) -> List[WhaleActivity]:
        """تشخیص فعالیت نهنگ‌ها"""
        activities = []

        df['vol_ma'] = df['volume'].rolling(20).mean()
        df['vol_ratio'] = df['volume'] / df['vol_ma']
        df['price_change'] = df['close'].pct_change().abs()

        whales = df[
            (df['vol_ratio'] > self.whale_volume_threshold) &
            (df['price_change'] > self.whale_price_impact)
        ].tail(20)

        for idx, candle in whales.iterrows():
            activity_type = 'accumulation' if candle['close'] > candle['open'] else 'distribution'

            vol_str = min(100, (candle['vol_ratio'] / self.whale_volume_threshold) * 50)
            price_str = min(100, (candle['price_change'] / self.whale_price_impact) * 50)
            strength = (vol_str + price_str) / 2

            activities.append(WhaleActivity(
                timestamp=int(idx.timestamp() * 1000),
                volume=candle['volume'],
                price=candle['close'],
                type=activity_type,
                strength=strength,
                impact=candle['price_change'] * 100
            ))

        logger.info(f"Whale Activity: {len(activities)} فعالیت نهنگ")
        return activities

    def generate_signal(
        self, symbol: str, tf: str, df: pd.DataFrame,
        structures: List[MarketStructure],
        obs: List[OrderBlock],
        fvgs: List[FairValueGap],
        zones: Dict,
        whales: List[WhaleActivity],
        liquidity: List[LiquidityZone],
        market: Dict
    ) -> Optional[TradingSignal]:
        """
        تولید سیگنال معاملاتی با منطق بهبود یافته
        شامل: اعتبارسنجی کامل، مدیریت ریسک، و تحلیل بازار
        """

        current_price = df['close'].iloc[-1]
        atr = df['atr'].iloc[-1] if 'atr' in df.columns else df['high'].iloc[-1] - df['low'].iloc[-1]

        long_score = 0
        short_score = 0
        reasons = []

        # ========== امتیازدهی LONG ==========

        # 1. Market Structure Analysis
        bullish_structures = [s for s in structures[-3:] if s.direction == 'bullish']
        if bullish_structures:
            last_bull = bullish_structures[-1]
            long_score += 20
            reasons.append(f"Structure: {last_bull.type} Bullish")

            if last_bull.type == 'CHoCH':
                long_score += 15
                reasons.append("CHoCH - تغییر روند به صعودی")

        # 2. Order Block Proximity
        bull_obs = [ob for ob in obs if ob.bias == 'bullish' and not ob.touched]
        if bull_obs:
            nearest = min(bull_obs, key=lambda x: abs(current_price - x.low))
            distance = abs(current_price - nearest.low) / current_price

            if distance < 0.02:
                long_score += 30
                reasons.append(f"OB Bullish قوی (قدرت: {nearest.strength:.0f}%)")

        # 3. Fair Value Gap
        bull_fvgs = [f for f in fvgs if f.bias == 'bullish' and not f.filled]
        if bull_fvgs:
            for fvg in bull_fvgs:
                if fvg.bottom <= current_price <= fvg.top:
                    long_score += 15
                    reasons.append("قیمت در FVG Bullish")
                    break

        # 4. Premium/Discount Analysis
        if zones.get('current_zone') == 'discount':
            long_score += 20
            reasons.append("Discount Zone - ناحیه خرید مناسب")

        # 5. Whale Activity Detection
        whale_acc = [w for w in whales[-5:] if w.type == 'accumulation']
        if len(whale_acc) >= 2:
            long_score += 15
            avg_strength = sum(w.strength for w in whale_acc) / len(whale_acc)
            reasons.append(f"Whale Accumulation: {len(whale_acc)} مورد (قدرت: {avg_strength:.0f}%)")

        # 6. Liquidity Zones
        buy_liq = [l for l in liquidity if l.type == 'buy' and abs(current_price - l.price) / current_price < 0.01]
        if buy_liq:
            long_score += 10
            reasons.append("نزدیک به Buy Liquidity Zone")

        # ========== امتیازدهی SHORT ==========

        # 1. Market Structure Analysis
        bearish_structures = [s for s in structures[-3:] if s.direction == 'bearish']
        if bearish_structures:
            last_bear = bearish_structures[-1]
            short_score += 20
            reasons.append(f"Structure: {last_bear.type} Bearish")

            if last_bear.type == 'CHoCH':
                short_score += 15
                reasons.append("CHoCH - تغییر روند به نزولی")

        # 2. Order Block Proximity
        bear_obs = [ob for ob in obs if ob.bias == 'bearish' and not ob.touched]
        if bear_obs:
            nearest = min(bear_obs, key=lambda x: abs(current_price - x.high))
            distance = abs(current_price - nearest.high) / current_price

            if distance < 0.02:
                short_score += 30
                reasons.append(f"OB Bearish قوی (قدرت: {nearest.strength:.0f}%)")

        # 3. Fair Value Gap
        bear_fvgs = [f for f in fvgs if f.bias == 'bearish' and not f.filled]
        if bear_fvgs:
            for fvg in bear_fvgs:
                if fvg.bottom <= current_price <= fvg.top:
                    short_score += 15
                    reasons.append("قیمت در FVG Bearish")
                    break

        # 4. Premium/Discount Analysis
        if zones.get('current_zone') == 'premium':
            short_score += 20
            reasons.append("Premium Zone - ناحیه فروش مناسب")

        # 5. Whale Activity Detection
        whale_dist = [w for w in whales[-5:] if w.type == 'distribution']
        if len(whale_dist) >= 2:
            short_score += 15
            avg_strength = sum(w.strength for w in whale_dist) / len(whale_dist)
            reasons.append(f"Whale Distribution: {len(whale_dist)} مورد (قدرت: {avg_strength:.0f}%)")

        # 6. Liquidity Zones
        sell_liq = [l for l in liquidity if l.type == 'sell' and abs(current_price - l.price) / current_price < 0.01]
        if sell_liq:
            short_score += 10
            reasons.append("نزدیک به Sell Liquidity Zone")

        # ========== تحلیل بازار کلی ==========
        if market:
            # BTC Trend
            btc_trend = market.get('btc_trend', 'neutral')
            if btc_trend == 'rising':
                long_score += 10
                reasons.append("BTC در روند صعودی")
            elif btc_trend == 'falling':
                short_score += 10
                reasons.append("BTC در روند نزولی")

            # USDT Dominance Effect
            usdt_d = market.get('usdt_dominance', 0)
            if usdt_d > 6:
                short_score += 10
                reasons.append(f"USDT.D بالا ({usdt_d:.1f}%) - Risk Off")

            # Market Phase Effect
            market_phase = market.get('market_phase', 'neutral')

            if market_phase == 'altseason':
                if symbol != 'BTC/USDT':
                    long_score += 10
                    reasons.append("Altseason Phase - فرصت خرید آلت‌کوین")

            elif market_phase == 'bear_market':
                short_score += 10
                reasons.append("Bear Market Phase - بازار نزولی")

            elif market_phase == 'btc_accumulation':
                if symbol == 'BTC/USDT':
                    long_score += 10
                    reasons.append("BTC Accumulation Phase")

        # ========== بررسی حداقل تفاوت امتیاز ==========
        min_score_diff = BOT_SETTINGS.get('min_score_difference', 25)

        if abs(long_score - short_score) < min_score_diff:
            logger.warning(f"⚠️ تضاد سیگنال شناسایی شد:")
            logger.warning(f"   LONG Score: {long_score}")
            logger.warning(f"   SHORT Score: {short_score}")
            logger.warning(f"   اختلاف: {abs(long_score - short_score)} (حداقل مورد نیاز: {min_score_diff})")
            logger.warning(f"   نتیجه: سیگنال واضحی وجود ندارد - منتظر ستاپ بهتر")
            return None

        # ========== تصمیم‌گیری نهایی ==========
        min_score = 60

        if long_score > short_score and long_score >= min_score:
            direction = 'LONG'
            confidence = min(95, long_score)
        elif short_score > long_score and short_score >= min_score:
            direction = 'SHORT'
            confidence = min(95, short_score)
        else:
            logger.info(f"امتیاز کافی برای سیگنال نیست:")
            logger.info(f"   LONG: {long_score}, SHORT: {short_score} (حداقل: {min_score})")
            return None

        # ========== محاسبه Stop Loss و Targets - اصلاح شده ==========
        entry = current_price
        sl_buffer_pct = BOT_SETTINGS.get('sl_buffer_percent', 0.5)

        if direction == 'LONG':
            swing_highs, swing_lows = self.detect_swing_points(df, 20)

            # محاسبه Stop Loss منطقی
            if len(swing_lows) > 0:
                sl = swing_lows.iloc[-1] * (1 - sl_buffer_pct / 100)
            else:
                sl = entry - atr * 2

            # ⬅️ تغییر: اعتبارسنجی SL بدون محدودیت 3%
            if sl >= entry:
                logger.warning("⚠️ SL نامعتبر برای LONG - استفاده از ATR")
                sl = entry - atr * 2

            # ⬅️ حذف شد: چک کردن محدودیت 3% و تنظیم مجدد SL
            # فقط محاسبه ریسک واقعی
            risk = entry - sl
            risk_pct = (risk / entry) * 100

            logger.info(f"📊 LONG Setup: Entry={entry:.8f}, SL={sl:.8f}, Risk={risk_pct:.2f}%")

            # محاسبه Targets
            targets = [
                entry + risk * 1.5,  # TP1: 1.5R
                entry + risk * 2.5,  # TP2: 2.5R
                entry + risk * 4.0   # TP3: 4.0R
            ]

        else:  # SHORT
            swing_highs, swing_lows = self.detect_swing_points(df, 20)

            # محاسبه Stop Loss منطقی
            if len(swing_highs) > 0:
                sl = swing_highs.iloc[-1] * (1 + sl_buffer_pct / 100)
            else:
                sl = entry + atr * 2

            # ⬅️ تغییر: اعتبارسنجی SL بدون محدودیت 3%
            if sl <= entry:
                logger.warning("⚠️ SL نامعتبر برای SHORT - استفاده از ATR")
                sl = entry + atr * 2

            # ⬅️ حذف شد: چک کردن محدودیت 3% و تنظیم مجدد SL
            # فقط محاسبه ریسک واقعی
            risk = sl - entry
            risk_pct = (risk / entry) * 100

            logger.info(f"📊 SHORT Setup: Entry={entry:.8f}, SL={sl:.8f}, Risk={risk_pct:.2f}%")

            # محاسبه Targets
            targets = [
                entry - risk * 1.5,  # TP1: 1.5R
                entry - risk * 2.5,  # TP2: 2.5R
                entry - risk * 4.0   # TP3: 4.0R
            ]

        # ========== اعتبارسنجی نهایی سیگنال ==========
        if direction == 'LONG':
            if sl >= entry:
                logger.error("❌ سیگنال LONG نامعتبر: SL بالاتر از Entry")
                logger.error(f"   Entry: {entry:.8f}, SL: {sl:.8f}")
                return None

            if any(tp <= entry for tp in targets):
                logger.error("❌ سیگنال LONG نامعتبر: یک یا چند Target پایین‌تر از Entry")
                logger.error(f"   Entry: {entry:.8f}, Targets: {[f'{t:.8f}' for t in targets]}")
                return None

        else:  # SHORT
            if sl <= entry:
                logger.error("❌ سیگنال SHORT نامعتبر: SL پایین‌تر از Entry")
                logger.error(f"   Entry: {entry:.8f}, SL: {sl:.8f}")
                return None

            if any(tp >= entry for tp in targets):
                logger.error("❌ سیگنال SHORT نامعتبر: یک یا چند Target بالاتر از Entry")
                logger.error(f"   Entry: {entry:.8f}, Targets: {[f'{t:.8f}' for t in targets]}")
                return None

        # ========== ایجاد شیء سیگنال ==========
        signal = TradingSignal(
            symbol=symbol,
            direction=direction,
            entry_price=entry,
            stop_loss=sl,
            targets=targets,
            confidence=confidence,
            timeframe=tf,
            reasons=reasons,
            market_context=market,
            timestamp=datetime.now()
        )

        # ========== گزارش‌دهی کامل ==========
        logger.info(f"\n{'='*70}")
        logger.info(f"✅ {direction} Signal Generated / سیگنال {direction} تولید شد")
        logger.info(f"{'='*70}")
        logger.info(f"📊 Symbol: {symbol}")
        logger.info(f"⏰ Timeframe: {tf}")
        logger.info(f"🎯 Confidence: {confidence:.0f}%")
        logger.info(f"💰 Entry: {entry:.8f}")
        logger.info(f"🛡️ Stop Loss: {sl:.8f} (Risk: {signal.risk_percent:.2f}%)")
        logger.info(f"📈 Risk/Reward: 1:{signal.risk_reward:.2f}")
        logger.info(f"\n🎯 Targets:")
        logger.info(f"   TP1: {targets[0]:.8f} (1.5R)")
        logger.info(f"   TP2: {targets[1]:.8f} (2.5R)")
        logger.info(f"   TP3: {targets[2]:.8f} (4.0R)")
        logger.info(f"\n📋 Reasons ({len(reasons)}):")
        for i, reason in enumerate(reasons, 1):
            logger.info(f"   {i}. {reason}")
        logger.info(f"\n💡 Position Size:")
        logger.info(f"   {signal.position_size_recommendation}")
        logger.info(f"{'='*70}\n")

        return signal

# ==================== بخش اصلاح شده: create_advanced_chart ====================

    def create_advanced_chart(
        self, symbol: str, df: pd.DataFrame, signal: TradingSignal,
        structures: List[MarketStructure],
        obs: List[OrderBlock],
        fvgs: List[FairValueGap],
        zones: Dict,
        whales: List[WhaleActivity],
        liquidity: List[LiquidityZone]
    ) -> str:
        """ایجاد نمودار با fallback به matplotlib"""

        # بررسی دسترسی به kaleido
        try:
            import kaleido
            has_kaleido = True
        except ImportError:
            has_kaleido = False
            logger.warning("⚠️ Kaleido در دسترس نیست - استفاده از matplotlib")

        if has_kaleido:
            try:
                # تلاش برای Plotly
                fig = make_subplots(
                    rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.05,
                    row_heights=[0.7, 0.3],
                    subplot_titles=(f'{symbol} - {signal.timeframe}', 'Volume')
                )

                # کندل‌ها
                fig.add_trace(
                    go.Candlestick(
                        x=df.index,
                        open=df['open'],
                        high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        name='Price',
                        increasing_line_color='#26a69a',
                        decreasing_line_color='#ef5350'
                    ),
                    row=1, col=1
                )

                # Order Blocks
                for ob in obs[:3]:
                    ob_time = pd.to_datetime(ob.time, unit='ms')
                    if ob_time not in df.index:
                        continue

                    color = 'rgba(8, 153, 129, 0.3)' if ob.bias == 'bullish' else 'rgba(242, 54, 69, 0.3)'

                    fig.add_shape(
                        type='rect',
                        x0=ob_time, x1=df.index[-1],
                        y0=ob.low, y1=ob.high,
                        fillcolor=color,
                        line=dict(color=color.replace('0.3', '0.8'), width=1),
                        row=1, col=1
                    )

                # Premium/Discount
                if zones:
                    fig.add_shape(
                        type='rect',
                        x0=df.index[0], x1=df.index[-1],
                        y0=zones['premium']['bottom'], y1=zones['premium']['top'],
                        fillcolor='rgba(244, 67, 54, 0.1)',
                        line=dict(width=0),
                        row=1, col=1
                    )

                    fig.add_shape(
                        type='rect',
                        x0=df.index[0], x1=df.index[-1],
                        y0=zones['discount']['bottom'], y1=zones['discount']['top'],
                        fillcolor='rgba(76, 175, 80, 0.1)',
                        line=dict(width=0),
                        row=1, col=1
                    )

                # Signal
                entry_color = '#00ff00' if signal.direction == 'LONG' else '#ff0000'
                fig.add_trace(
                    go.Scatter(
                        x=[df.index[-1]], y=[signal.entry_price],
                        mode='markers',
                        marker=dict(size=20, color=entry_color, symbol='star'),
                        name=f'{signal.direction} Entry'
                    ),
                    row=1, col=1
                )

                # SL & TP
                fig.add_shape(
                    type='line',
                    x0=df.index[0], x1=df.index[-1],
                    y0=signal.stop_loss, y1=signal.stop_loss,
                    line=dict(color='red', width=2, dash='dash'),
                    row=1, col=1
                )

                for i, tp in enumerate(signal.targets):
                    fig.add_shape(
                        type='line',
                        x0=df.index[0], x1=df.index[-1],
                        y0=tp, y1=tp,
                        line=dict(color=['#ffeb3b', '#ff9800', '#4caf50'][i], width=2, dash='dot'),
                        row=1, col=1
                    )

                # Volume
                colors = ['#26a69a' if c >= o else '#ef5350' for c, o in zip(df['close'], df['open'])]
                fig.add_trace(
                    go.Bar(x=df.index, y=df['volume'], marker=dict(color=colors), name='Volume'),
                    row=2, col=1
                )

                # Layout
                title = f"{signal.direction} {symbol} | {signal.timeframe} | {signal.confidence:.0f}%"
                fig.update_layout(
                    title=title,
                    template='plotly_dark',
                    height=800,
                    xaxis_rangeslider_visible=False
                )

                chart_path = f"{BASE_DIR}/charts/{symbol.replace('/', '_')}_{signal.timeframe}_{int(time.time())}.png"
                fig.write_image(chart_path, width=1600, height=800, engine='kaleido')
                logger.info(f"✅ نمودار Plotly ذخیره شد: {chart_path}")
                return chart_path

            except Exception as e:
                logger.warning(f"خطا در Plotly: {e} - استفاده از matplotlib")

        # Fallback به matplotlib
        return self.create_fallback_chart(symbol, df, signal, obs, zones)


    def create_fallback_chart(self, symbol: str, df: pd.DataFrame, signal: TradingSignal,
                            obs: List[OrderBlock], zones: Dict) -> str:
        """نمودار matplotlib (همیشه کار می‌کند)"""
        plt.style.use('dark_background')
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), height_ratios=[3, 1], sharex=True)

        # کندل‌ها
        colors = ['#26a69a' if c >= o else '#ef5350' for c, o in zip(df['close'], df['open'])]

        for i in range(len(df)):
            ax1.plot([i, i], [df['low'].iloc[i], df['high'].iloc[i]], color=colors[i], linewidth=1)
            body_h = abs(df['close'].iloc[i] - df['open'].iloc[i])
            body_b = min(df['open'].iloc[i], df['close'].iloc[i])
            ax1.add_patch(Rectangle((i-0.3, body_b), 0.6, body_h,
                                    facecolor=colors[i], edgecolor=colors[i], alpha=0.8))

        # Order Blocks
        for ob in obs[:3]:
            ob_time = pd.to_datetime(ob.time, unit='ms')
            if ob_time in df.index:
                ob_idx = df.index.get_loc(ob_time)
                color = '#089981' if ob.bias == 'bullish' else '#f23645'
                ax1.add_patch(Rectangle((ob_idx, ob.low), len(df)-ob_idx, ob.high-ob.low,
                                        facecolor=color, alpha=0.2, edgecolor=color, linewidth=1))

        # Signal
        current_idx = len(df) - 1
        entry_color = '#00ff00' if signal.direction == 'LONG' else '#ff0000'
        ax1.scatter(current_idx, signal.entry_price, color=entry_color, marker='*',
                s=400, zorder=10, edgecolors='white', linewidths=2)

        # SL & TP
        ax1.axhline(signal.stop_loss, color='red', linestyle='--', linewidth=2, alpha=0.7, label='SL')
        for i, tp in enumerate(signal.targets):
            ax1.axhline(tp, color=['#ffeb3b', '#ff9800', '#4caf50'][i],
                    linestyle='-.', linewidth=2, alpha=0.7, label=f'TP{i+1}')

        ax1.set_ylabel('Price', fontsize=11)
        ax1.grid(True, alpha=0.2)
        ax1.legend(loc='upper left')

        # Volume
        ax2.bar(range(len(df)), df['volume'], color=colors, alpha=0.6)
        ax2.set_ylabel('Volume', fontsize=11)
        ax2.grid(True, alpha=0.2)

        title = f"{signal.direction} {symbol} | {signal.timeframe} | {signal.confidence:.0f}% | R:R 1:{signal.risk_reward:.1f}"
        fig.suptitle(title, fontsize=14, weight='bold')

        chart_path = f"{BASE_DIR}/charts/{symbol.replace('/', '_')}_{signal.timeframe}_{int(time.time())}.png"
        plt.savefig(chart_path, dpi=120, bbox_inches='tight', facecolor='#0d1117')
        plt.close()

        logger.info(f"✅ نمودار matplotlib ذخیره شد: {chart_path}")
        return chart_path

    def analyze_symbol(self, symbol: str, timeframes: List[str] = ['1h', '4h']) -> List[TradingSignal]:
        """
        تحلیل کامل یک ارز در تایم‌فریم‌های مختلف
        با مدیریت کامل حافظه و منابع
        """
        logger.info(f"\n{'='*70}")
        logger.info(f"شروع تحلیل {symbol}")
        logger.info(f"{'='*70}")

        signals = []

        # دریافت وضعیت بازار
        market = self.fetch_market_dominance()

        for tf in timeframes:
            logger.info(f"\n{'---'*20}")
            logger.info(f"تحلیل Timeframe: {tf}")
            logger.info(f"{'---'*20}")

            try:
                # دریافت داده
                df = self.fetch_ohlcv_with_cache(symbol, tf, limit=500)

                if df is None or len(df) < 100:
                    logger.warning(f"داده کافی برای تحلیل نیست: {symbol} در {tf}")
                    logger.warning(f"تعداد کندل: {len(df) if df is not None else 0}")
                    continue

                # محاسبه اندیکاتورها
                logger.info("محاسبه اندیکاتورها...")
                df['atr'] = ta.volatility.average_true_range(
                    df['high'], df['low'], df['close'], window=14
                )

                # تحلیل‌های Smart Money Concepts
                logger.info("تحلیل Market Structure...")
                structures = self.detect_market_structure(df, tf)

                logger.info("تحلیل Order Blocks...")
                obs = self.detect_order_blocks(df, structures)

                logger.info("تحلیل Fair Value Gaps...")
                fvgs = self.detect_fvg(df)

                logger.info("محاسبه Premium/Discount Zones...")
                zones = self.calculate_zones(df)

                logger.info("تشخیص Whale Activity...")
                whales = self.detect_whale_activity(df)

                logger.info("تحلیل Liquidity Zones...")
                liquidity = self.detect_liquidity_zones(df)

                # تولید سیگنال
                logger.info("تولید سیگنال معاملاتی...")
                signal = self.generate_signal(
                    symbol, tf, df, structures, obs, fvgs,
                    zones, whales, liquidity, market
                )

                if signal:
                    logger.info(f"✅ سیگنال {signal.direction} با اطمینان {signal.confidence:.0f}% تولید شد")

                    # ایجاد نمودار
                    logger.info("ایجاد نمودار تحلیلی...")
                    chart_path = self.create_advanced_chart(
                        symbol, df, signal, structures, obs, fvgs,
                        zones, whales, liquidity
                    )
                    signal.chart_path = chart_path

                    signals.append(signal)
                else:
                    logger.info(f"ℹ️ سیگنالی برای {symbol} در {tf} تولید نشد")

            except Exception as e:
                logger.error(f"خطا در تحلیل {symbol} در {tf}: {e}")
                logger.error(traceback.format_exc())

            finally:
                # پاکسازی حافظه
                if 'df' in locals():
                    del df
                gc.collect()

                # بررسی منابع
                resources = monitor_resources()
                if resources:
                    logger.info(f"وضعیت منابع: RAM={resources.get('memory_percent', 0):.1f}%, "
                              f"Cache Items={resources.get('cache_items', 0)}")

                # پاکسازی فایل‌های قدیمی
                cleanup_drive_files()

        # خلاصه نهایی
        logger.info(f"\n{'='*70}")
        logger.info(f"تحلیل {symbol} تکمیل شد")
        logger.info(f"تعداد سیگنال‌های تولید شده: {len(signals)}")
        if signals:
            for sig in signals:
                logger.info(f"  - {sig.timeframe}: {sig.direction} ({sig.confidence:.0f}%)")
        logger.info(f"{'='*70}\n")

        return signals


# ==================== کلاس ربات تلگرام ====================
# ==================== کلاس ربات تلگرام - نسخه کامل ====================

import asyncio
from collections import defaultdict

class TelegramBot:
    """ربات تلگرام با قابلیت‌های کامل"""

    def __init__(self, token: str, channel_id: str):
        self.token = token
        self.channel_id = channel_id
        self.analyzer = SmartMoneyWhaleHunter()
        self.app = None

        if not TELEGRAM_AVAILABLE:
            logger.warning("⚠️ تلگرام غیرفعال - فقط تحلیل مستقیم امکان‌پذیر است")

    def log_user_activity(self, user_id: int, username: str, symbol: str, action: str):
        """ثبت فعالیت کاربران"""
        try:
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            log_entry = f"[{timestamp}] User: {username} (ID: {user_id}) | Action: {action} | Symbol: {symbol}"

            # لاگ در فایل
            logger.info(f"👤 {log_entry}")

            # لاگ در فایل جداگانه برای آمار
            stats_file = f"{BASE_DIR}/logs/user_stats.log"
            with open(stats_file, 'a', encoding='utf-8') as f:
                f.write(log_entry + '\n')

        except Exception as e:
            logger.error(f"خطا در ثبت لاگ کاربر: {e}")

    async def start_command(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """دستور /start"""
        msg = """
🐋 **Smart Money Whale Hunter Bot**
**Professional Trading Signal System**

**📋 دستورات:**
• `/analyze BTC/USDT` - تحلیل ارز (1h & 4h)
• `/analyze ETH/USDT 1h` - تحلیل تایم‌فریم خاص
• `/analyze SOL/USDT 4h` - تحلیل تایم‌فریم خاص
• `/help` - راهنما
• `/stats` - آمار استفاده (فقط ادمین)

**✨ ویژگی‌ها:**
✅ Smart Money Concepts (SMC)
✅ تشخیص Order Blocks
✅ Fair Value Gaps (FVG)
✅ Market Structure (BOS/CHoCH)
✅ Whale Activity Detection
✅ USDT Dominance Analysis
✅ Market Phase Detection
✅ مدیریت ریسک حرفه‌ای (حداکثر 3%)
✅ اعتبارسنجی کامل سیگنال‌ها
✅ نمودارهای تحلیلی پیشرفته

**⚠️ توجه:**
این ربات یک ابزار کمکی برای تحلیل است.
همیشه تحقیقات شخصی انجام دهید و مدیریت ریسک را رعایت کنید.

توسعه: حسین فایننس 🤖
نسخه: 2.1 Enhanced
        """
        await update.message.reply_text(msg, parse_mode='Markdown')

    async def help_command(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """دستور /help"""
        msg = """
📚 راهنمای کامل استفاده

🔍 نحوه تحلیل:
- /analyze BTC/USDT - تحلیل در 1h و 4h
- /analyze ETH/USDT 1h - فقط 1h
- /analyze BNB/USDT 4h - فقط 4h

⏱️ تایم‌فریم‌های پشتیبانی شده:
- 15m - پانزده دقیقه (اسکالپ)
- 1h - یک ساعت (سوئینگ کوتاه)
- 4h - چهار ساعت (سوئینگ)
- 1d - روزانه (بلندمدت)

📊 توضیح المان‌های سیگنال:

1. Entry (ورود):
قیمت پیشنهادی ورود به معامله

2. Stop Loss (حد ضرر):
نقطه خروج در صورت خلاف پیش‌بینی
بر اساس استراتژی SMC تعیین می‌شود

3. Take Profit (اهداف سود):
• TP1: هدف اول (1.5R)
• TP2: هدف دوم (2.5R)
• TP3: هدف سوم (4.0R)

4. Confidence (اطمینان):
درصد اطمینان سیگنال (60-95%)
بالاتر = قوی‌تر

5. R:R (Risk/Reward):
نسبت ریسک به ریوارد
حداقل 1:1.5 توصیه می‌شود

📖 اصطلاحات SMC:

- BOS (Break of Structure):
شکست ساختار - ادامه روند

- CHoCH (Change of Character):
تغییر روند - معکوس شدن

- OB (Order Block):
بلوک سفارش - ناحیه ورود نهنگ‌ها

- FVG (Fair Value Gap):
شکاف قیمتی - ناحیه پرشدنی

- Premium Zone:
ناحیه گران - مناسب فروش

- Discount Zone:
ناحیه ارزان - مناسب خرید

🌐 شاخص‌های بازار:

- BTC Dominance (BTC.D):
درصد سهم بیت‌کوین از بازار
بالا = قدرت BTC، پایین = Altseason

- USDT Dominance (USDT.D):
درصد سهم تتر از بازار
بالا (>6%) = خروج پول (Risk-Off)
پایین (<6%) = ورود پول (Risk-On)

- Market Phase:
- Altseason: فرصت آلت‌کوین
- Bear Market: بازار نزولی
- BTC Accumulation: جمع‌آوری بیت‌کوین
- Neutral: بازار خنثی

💡 نکات مهم:

1. مدیریت ریسک:
حداکثر 1-2% سرمایه در هر معامله

2. Position Sizing:
حجم = (سرمایه × ریسک%) / (فاصله تا SL)

3. تایید چندگانه:
سیگنال قوی = همگرایی چند فاکتور

4. Stop Loss:
NEVER معامله بدون استاپ لاس

5. Take Profit:
می‌توانید پله‌ای (TP1: 50%, TP2: 30%, TP3: 20%) خارج شوید

⚠️ هشدارها:

❌ این ربات مشاور مالی نیست
❌ DYOR (تحقیق شخصی) الزامی
❌ سرمایه‌ای که توان از دست دادنش را ندارید وارد نکنید
❌ احساسات را کنار بگذارید
❌ به یک سیگنال اکتفا نکنید

🆘 پشتیبانی:
برای گزارش باگ یا پیشنهاد: @FINANCE_HOSSEIN

موفق باشید! 🚀
        """
        await update.message.reply_text(msg)

    async def stats_command(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """نمایش آمار استفاده (فقط برای ادمین)"""
        user = update.effective_user
        user_id = user.id
        username = user.username

        # بررسی دسترسی ادمین
        if user_id not in ADMIN_USERS and (not username or f"@{username}" not in ADMIN_USERS):
            await update.message.reply_text("❌ شما دسترسی به این دستور ندارید.")
            return

        try:
            stats_file = f"{BASE_DIR}/logs/user_stats.log"

            if not os.path.exists(stats_file):
                await update.message.reply_text("ℹ️ هنوز آماری ثبت نشده است.")
                return

            # خواندن و تحلیل لاگ‌ها
            with open(stats_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()

            # آمارگیری
            total_requests = len(lines)
            users = set()
            symbols = defaultdict(int)
            timeframes = defaultdict(int)

            for line in lines:
                # استخراج اطلاعات از لاگ
                if 'User:' in line and 'Symbol:' in line:
                    # استخراج کاربر
                    try:
                        user_part = line.split('User:')[1].split('(ID:')[0].strip()
                        users.add(user_part)
                    except:
                        pass

                    # استخراج symbol
                    try:
                        symbol_part = line.split('Symbol:')[1].strip()
                        if '|' in symbol_part:
                            symbol_part = symbol_part.split('|')[0].strip()
                        symbols[symbol_part] += 1
                    except:
                        pass

                    # استخراج timeframe
                    try:
                        if 'analyze_' in line:
                            tf_part = line.split('analyze_')[1].strip()
                            if '|' in tf_part:
                                tf_part = tf_part.split('|')[0].strip()
                            timeframes[tf_part] += 1
                    except:
                        pass

            # ساخت پیام آمار
            msg = "📊 **آمار استفاده از ربات**\n"
            msg += "═" * 30 + "\n\n"

            msg += f"👥 **تعداد کاربران:** {len(users)}\n"
            msg += f"📈 **تعداد درخواست‌ها:** {total_requests}\n\n"

            # محبوب‌ترین ارزها
            if symbols:
                msg += "💎 **محبوب‌ترین ارزها:**\n"
                sorted_symbols = sorted(symbols.items(), key=lambda x: x[1], reverse=True)
                for i, (sym, count) in enumerate(sorted_symbols[:10], 1):
                    percentage = (count / total_requests) * 100
                    msg += f"{i}. `{sym}`: {count} بار ({percentage:.1f}%)\n"
                msg += "\n"

            # محبوب‌ترین تایم‌فریم‌ها
            if timeframes:
                msg += "⏰ **محبوب‌ترین تایم‌فریم‌ها:**\n"
                sorted_tfs = sorted(timeframes.items(), key=lambda x: x[1], reverse=True)
                for tf, count in sorted_tfs:
                    percentage = (count / total_requests) * 100
                    msg += f"• `{tf}`: {count} بار ({percentage:.1f}%)\n"
                msg += "\n"

            # آخرین درخواست‌ها
            msg += "📝 **آخرین 10 درخواست:**\n"
            recent_lines = [l for l in lines if 'User:' in l and 'Symbol:' in l][-10:]

            for line in recent_lines:
                try:
                    # استخراج زمان
                    timestamp = line.split('[')[1].split(']')[0].split(' ')[1]  # فقط ساعت

                    # استخراج کاربر
                    user_part = line.split('User:')[1].split('(ID:')[0].strip()
                    if len(user_part) > 15:
                        user_part = user_part[:12] + "..."

                    # استخراج symbol
                    symbol_part = line.split('Symbol:')[1].strip()
                    if '|' in symbol_part:
                        symbol_part = symbol_part.split('|')[0].strip()

                    msg += f"• `{timestamp}` - {user_part} → `{symbol_part}`\n"
                except:
                    continue

            # اطلاعات سیستم
            msg += "\n📊 **وضعیت سیستم:**\n"
            resources = monitor_resources()
            if resources:
                msg += f"• حافظه: {resources.get('memory_percent', 0):.1f}%\n"
                msg += f"• کش: {resources.get('cache_items', 0)} آیتم\n"

            # آمار صرافی‌ها
            msg += "\n🏦 **وضعیت صرافی‌ها:**\n"
            for ex_id, state in exchange_data.items():
                success_rate = 0
                total = state['success_count'] + state['fail_count']
                if total > 0:
                    success_rate = (state['success_count'] / total) * 100

                status_icon = "✅" if not state['in_cooldown'] else "⏳"
                msg += f"{status_icon} `{ex_id}`: {success_rate:.0f}% موفق\n"

            msg += f"\n🕐 آخرین به‌روزرسانی: {datetime.now().strftime('%H:%M:%S')}"

            await update.message.reply_text(msg, parse_mode='Markdown')
            logger.info(f"📊 آمار برای ادمین {username} نمایش داده شد")

        except Exception as e:
            logger.error(f"خطا در stats_command: {e}")
            logger.error(traceback.format_exc())
            await update.message.reply_text(f"❌ خطا در نمایش آمار: {str(e)[:100]}")

    async def analyze_command(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        """دستور /analyze"""

        # استخراج اطلاعات کاربر
        user = update.effective_user
        user_id = user.id
        username = user.username or user.first_name or "Unknown"

        if not context.args:
            await update.message.reply_text(
                "❌ **فرمت نادرست**\n\n"
                "**مثال‌های صحیح:**\n"
                "`/analyze BTC/USDT`\n"
                "`/analyze ETH/USDT 1h`\n"
                "`/analyze SOL/USDT 4h`\n\n"
                "**تایم‌فریم‌های معتبر:** 15m, 1h, 4h, 1d",
                parse_mode='Markdown'
            )
            return

        # استخراج symbol
        symbol = context.args[0].upper()
        if '/' not in symbol:
            symbol = f"{symbol}/USDT"

        # استخراج timeframe
        timeframes = ['1h', '4h']  # پیش‌فرض
        if len(context.args) > 1:
            tf = context.args[1].lower()
            if tf in ['15m', '1h', '4h', '1d']:
                timeframes = [tf]
            else:
                await update.message.reply_text(
                    f"❌ **تایم‌فریم نامعتبر:** `{tf}`\n\n"
                    "**تایم‌فریم‌های معتبر:**\n"
                    "• `15m` - پانزده دقیقه\n"
                    "• `1h` - یک ساعت\n"
                    "• `4h` - چهار ساعت\n"
                    "• `1d` - روزانه",
                    parse_mode='Markdown'
                )
                return

        # ثبت لاگ فعالیت کاربر
        self.log_user_activity(user_id, username, symbol, f"analyze_{','.join(timeframes)}")

        # پیام شروع تحلیل
        status_msg = await update.message.reply_text(
            f"🔍 **در حال تحلیل {symbol}...**\n\n"
            f"⏱️ تایم‌فریم: {', '.join(timeframes)}\n"
            f"⏳ لطفاً صبر کنید...",
            parse_mode='Markdown'
        )

        try:
            # انجام تحلیل
            signals = self.analyzer.analyze_symbol(symbol, timeframes)

            # حذف پیام وضعیت
            await status_msg.delete()

            if not signals:
                await update.message.reply_text(
                    f"ℹ️ **هیچ سیگنالی برای {symbol} یافت نشد**\n\n"
                    "**دلایل احتمالی:**\n"
                    "• شرایط بازار مناسب نیست\n"
                    "• تضاد بین سیگنال‌های LONG و SHORT\n"
                    "• امتیاز کمتر از 60%\n"
                    "• ستاپ کامل تشکیل نشده\n\n"
                    "💡 **توصیه:** منتظر ستاپ واضح‌تر بمانید",
                    parse_mode='Markdown'
                )
                return

            # ارسال هر سیگنال
            for signal in signals:
                await self.send_signal_message(update, signal)
                await asyncio.sleep(1)  # تاخیر بین پیام‌ها

            # ثبت لاگ سیگنال‌های تولید شده
            logger.info(f"📤 {len(signals)} سیگنال برای {username} ارسال شد")

        except Exception as e:
            logger.error(f"خطا در analyze_command: {e}")
            logger.error(traceback.format_exc())

            await update.message.reply_text(
                f"❌ **خطا در تحلیل {symbol}**\n\n"
                f"**جزئیات:** `{str(e)[:150]}`\n\n"
                "لطفاً دوباره تلاش کنید یا با پشتیبانی تماس بگیرید.",
                parse_mode='Markdown'
            )

    async def send_signal_message(self, update: Update, signal: TradingSignal):

        def escape_markdown_v2(text: str) -> str:
            """
            Escape تمام کاراکترهای خاص برای MarkdownV2
            کاراکترهای رزرو شده در تلگرام:
            _ * [ ] ( ) ~ ` > # + - = | { } . !
            """
            escape_chars = r'_*[]()~`>#+=|{}.!-'
            for char in escape_chars:
                text = text.replace(char, '\\' + char)
            return text

        # تعیین ایموجی
        if signal.direction == "LONG":
            direction_emoji = "🟢"
            direction_icon = "📈"
        else:
            direction_emoji = "🔴"
            direction_icon = "📉"

        # ساخت پیام اصلی (بدون escape - خودمان format می‌کنیم)
        header = f"{direction_emoji} *{signal.direction} SIGNAL* {direction_icon}"
        separator = "━━━━━━━━━━━━━━━━━━━━━"

        # اطلاعات اصلی
        info_lines = [
            f"*💎 Symbol:* `{signal.symbol}`",
            f"*⏰ Timeframe:* `{signal.timeframe}`",
            f"*🎯 Confidence:* `{signal.confidence:.0f}%`",
            f"*📊 Risk/Reward:* `1:{signal.risk_reward:.2f}`",
            f"*⚠️ Risk:* `{signal.risk_percent:.2f}%`"
        ]

        # Entry
        entry_section = [
            "*📍 ENTRY ZONE*",
            f"💰 *Entry Price:* `{signal.entry_price:.8f}`"
        ]

        # Stop Loss
        sl_section = [
            "*🛡️ STOP LOSS*",
            f"🚫 *Stop Loss:* `{signal.stop_loss:.8f}`"
        ]

        # Take Profits
        tp_section = [
            "*🎯 TAKE PROFIT TARGETS*",
            f"🥇 *TP1 \\(1\\.5R\\):* `{signal.targets[0]:.8f}`",
            f"🥈 *TP2 \\(2\\.5R\\):* `{signal.targets[1]:.8f}`",
            f"🥉 *TP3 \\(4\\.0R\\):* `{signal.targets[2]:.8f}`"
        ]

        # ترکیب بخش‌های بالا
        msg_parts = [
            header,
            separator,
            "",
            "\n".join(info_lines),
            "",
            separator,
            "\n".join(entry_section),
            "",
            separator,
            "\n".join(sl_section),
            "",
            separator,
            "\n".join(tp_section),
            "",
            separator,
            "*📋 SIGNAL REASONS*",
            separator
        ]

        msg = "\n".join(msg_parts)

        # اضافه کردن دلایل
        reasons_text = ""
        for i, reason in enumerate(signal.reasons, 1):
            # پاکسازی و escape کردن هر دلیل
            clean_reason = reason.replace('✅', '✔').replace('⚡', '⚠').replace('📦', '▪').replace('🔊', '▪')
            escaped_reason = escape_markdown_v2(clean_reason)
            reasons_text += f"{i}\\. {escaped_reason}\n"

        msg += "\n" + reasons_text

        # Market Context
        if signal.market_context:
            msg += f"\n{separator}\n"
            msg += f"*🌍 MARKET CONTEXT*\n"
            msg += f"{separator}\n"

            btc_d = signal.market_context.get('btc_dominance', 0)
            btc_trend = signal.market_context.get('btc_trend', 'neutral')
            usdt_d = signal.market_context.get('usdt_dominance', 0)
            market_phase = signal.market_context.get('market_phase', 'neutral')

            # Format با escape
            btc_d_str = f"{btc_d:.2f}".replace('.', '\\.')
            usdt_d_str = f"{usdt_d:.2f}".replace('.', '\\.')
            btc_trend_escaped = escape_markdown_v2(btc_trend)
            phase_escaped = escape_markdown_v2(market_phase.replace('_', ' '))

            msg += f"• *BTC\\.D:* {btc_d_str}% \\({btc_trend_escaped}\\)\n"
            msg += f"• *USDT\\.D:* {usdt_d_str}%\n"
            msg += f"• *Phase:* {phase_escaped}\n"

            if 'total_mcap' in signal.market_context:
                mcap = signal.market_context['total_mcap']
                mcap_str = f"{mcap/1e12:.2f}".replace('.', '\\.')
                msg += f"• *Total MCap:* ${mcap_str}T\n"

        # Position Size
        msg += f"\n{separator}\n"
        msg += f"*💡 POSITION SIZE*\n"
        msg += f"{separator}\n"
        msg += f"با ریسک 1% سرمایه:\n"
        msg += f"`Position = \\(Capital × 0\\.01\\) / Distance\\_to\\_SL`\n"

        # هشدارها
        msg += f"\n{separator}\n"
        msg += f"*⚠️ IMPORTANT NOTES*\n"
        msg += f"{separator}\n"
        msg += f"• همیشه Stop Loss را رعایت کنید\n"
        msg += f"• حداکثر 1\\-2% سرمایه در هر معامله\n"
        msg += f"• DYOR \\- تحقیقات شخصی انجام دهید\n"
        msg += f"• کانال: @finance\\_hossein\n"

        # تاریخ
        timestamp_str = signal.timestamp.strftime('%Y-%m-%d %H:%M:%S')
        timestamp_escaped = escape_markdown_v2(timestamp_str)
        msg += f"\n📅 {timestamp_escaped} UTC"

        # ارسال پیام
        try:
            await update.message.reply_text(msg, parse_mode='MarkdownV2')
            logger.info("✅ پیام سیگنال با موفقیت ارسال شد")

        except Exception as e:
            logger.error(f"خطا در ارسال پیام MarkdownV2: {e}")

            # Fallback: ارسال متن ساده بدون فرمت
            try:
                simple_msg = f"""
    {direction_emoji} {signal.direction} SIGNAL {direction_icon}
    ━━━━━━━━━━━━━━━━━━━━━

    💎 Symbol: {signal.symbol}
    ⏰ Timeframe: {signal.timeframe}
    🎯 Confidence: {signal.confidence:.0f}%
    📊 Risk/Reward: 1:{signal.risk_reward:.2f}
    ⚠️ Risk: {signal.risk_percent:.2f}%

    ━━━━━━━━━━━━━━━━━━━━━
    📍 ENTRY ZONE
    ━━━━━━━━━━━━━━━━━━━━━
    💰 Entry: {signal.entry_price:.8f}

    ━━━━━━━━━━━━━━━━━━━━━
    🛡️ STOP LOSS
    ━━━━━━━━━━━━━━━━━━━━━
    🚫 Stop Loss: {signal.stop_loss:.8f}

    ━━━━━━━━━━━━━━━━━━━━━
    🎯 TAKE PROFIT TARGETS
    ━━━━━━━━━━━━━━━━━━━━━
    🥇 TP1 (1.5R): {signal.targets[0]:.8f}
    🥈 TP2 (2.5R): {signal.targets[1]:.8f}
    🥉 TP3 (4.0R): {signal.targets[2]:.8f}

    ━━━━━━━━━━━━━━━━━━━━━
    📋 SIGNAL REASONS ({len(signal.reasons)} دلیل)
    ━━━━━━━━━━━━━━━━━━━━━
    """
                # اضافه کردن دلایل
                for i, reason in enumerate(signal.reasons, 1):
                    simple_msg += f"{i}. {reason}\n"

                # Market Context
                if signal.market_context:
                    simple_msg += f"\n━━━━━━━━━━━━━━━━━━━━━\n"
                    simple_msg += f"🌍 MARKET CONTEXT\n"
                    simple_msg += f"━━━━━━━━━━━━━━━━━━━━━\n"

                    btc_d = signal.market_context.get('btc_dominance', 0)
                    btc_trend = signal.market_context.get('btc_trend', 'neutral')
                    usdt_d = signal.market_context.get('usdt_dominance', 0)
                    market_phase = signal.market_context.get('market_phase', 'neutral')

                    simple_msg += f"• BTC.D: {btc_d:.2f}% ({btc_trend})\n"
                    simple_msg += f"• USDT.D: {usdt_d:.2f}%\n"
                    simple_msg += f"• Phase: {market_phase}\n"

                # Position Size
                simple_msg += f"\n━━━━━━━━━━━━━━━━━━━━━\n"
                simple_msg += f"💡 POSITION SIZE\n"
                simple_msg += f"━━━━━━━━━━━━━━━━━━━━━\n"
                simple_msg += f"با ریسک 1% سرمایه محاسبه کنید\n"
                simple_msg += f"Position = (Capital × 0.01) / Distance_to_SL\n"

                # هشدارها
                simple_msg += f"\n━━━━━━━━━━━━━━━━━━━━━\n"
                simple_msg += f"⚠️ IMPORTANT NOTES\n"
                simple_msg += f"━━━━━━━━━━━━━━━━━━━━━\n"
                simple_msg += f"• همیشه Stop Loss را رعایت کنید\n"
                simple_msg += f"• حداکثر 1-2% سرمایه در هر معامله\n"
                simple_msg += f"• DYOR - تحقیقات شخصی انجام دهید\n"
                simple_msg += f"• کانال: @finance_hossein\n"

                simple_msg += f"\n📅 {signal.timestamp.strftime('%Y-%m-%d %H:%M:%S')} UTC"

                await update.message.reply_text(simple_msg)
                logger.info("✅ پیام ساده (fallback) ارسال شد")

            except Exception as e2:
                logger.error(f"خطا در ارسال پیام ساده: {e2}")

        # ارسال نمودار
        if signal.chart_path and os.path.exists(signal.chart_path):
            try:
                with open(signal.chart_path, 'rb') as photo:
                    caption = f"{signal.symbol} | {signal.timeframe} | {signal.direction} | {signal.confidence:.0f}%"
                    await update.message.reply_photo(photo=photo, caption=caption)
                logger.info(f"✅ نمودار ارسال شد: {signal.chart_path}")
            except Exception as e:
                logger.error(f"خطا در ارسال نمودار: {e}")
                logger.error(traceback.format_exc())
        else:
            logger.warning(f"⚠️ فایل نمودار یافت نشد یا وجود ندارد: {signal.chart_path}")

    def run(self):
        """اجرای ربات تلگرام"""
        if not TELEGRAM_AVAILABLE:
            logger.error("❌ کتابخانه تلگرام در دسترس نیست")
            return

        logger.info("🚀 راه‌اندازی ربات تلگرام...")

        try:
            self.app = Application.builder().token(self.token).build()

            # اضافه کردن دستورات
            self.app.add_handler(CommandHandler("start", self.start_command))
            self.app.add_handler(CommandHandler("help", self.help_command))
            self.app.add_handler(CommandHandler("analyze", self.analyze_command))
            self.app.add_handler(CommandHandler("stats", self.stats_command))  # ⬅️ اضافه شد

            logger.info("✅ ربات آماده است")
            logger.info("📱 برای شروع دستور /start را در تلگرام بزنید")

            # شروع polling
            self.app.run_polling(allowed_updates=Update.ALL_TYPES)

        except Exception as e:
            logger.error(f"❌ خطا در اجرای ربات: {e}")
            logger.error(traceback.format_exc())


# ==================== تابع اصلی ====================

async def main():
    """
    تابع اصلی برای تست و اجرای اولیه
    """
    logger.info("="*70)
    logger.info("🚀 Smart Money Whale Hunter - Professional Edition v2.1")
    logger.info("="*70)

    # تست با BTC
    test_symbol = 'BTC/USDT'
    test_timeframes = ['1h', '4h']

    logger.info(f"\n🧪 تست سیستم با {test_symbol}")
    logger.info(f"⏱️ تایم‌فریم‌ها: {', '.join(test_timeframes)}\n")

    # ایجاد analyzer
    hunter = SmartMoneyWhaleHunter()

    try:
        # تحلیل
        signals = hunter.analyze_symbol(test_symbol, test_timeframes)

        # گزارش نتایج
        if signals:
            logger.info(f"\n{'='*70}")
            logger.info(f"✅ تست موفق - {len(signals)} سیگنال تولید شد")
            logger.info(f"{'='*70}")

            for i, signal in enumerate(signals, 1):
                logger.info(f"\n📊 سیگنال #{i}:")
                logger.info(f"   Symbol: {signal.symbol}")
                logger.info(f"   Direction: {signal.direction}")
                logger.info(f"   Timeframe: {signal.timeframe}")
                logger.info(f"   Confidence: {signal.confidence:.0f}%")
                logger.info(f"   Risk/Reward: 1:{signal.risk_reward:.2f}")
                logger.info(f"   Risk: {signal.risk_percent:.2f}%")
                logger.info(f"   Entry: {signal.entry_price:.8f}")
                logger.info(f"   Stop Loss: {signal.stop_loss:.8f}")
                logger.info(f"   TP1: {signal.targets[0]:.8f}")
        else:
            logger.info(f"\n{'='*70}")
            logger.info(f"ℹ️ هیچ سیگنالی برای {test_symbol} تولید نشد")
            logger.info(f"{'='*70}")
            logger.info("این طبیعی است - سیستم فقط ستاپ‌های با کیفیت بالا را می‌پذیرد")

    except Exception as e:
        logger.error(f"\n{'='*70}")
        logger.error(f"❌ خطا در تست: {e}")
        logger.error(f"{'='*70}")
        logger.error(traceback.format_exc())

    logger.info(f"\n{'='*70}")
    logger.info("✅ تست اولیه تکمیل شد")
    logger.info("🤖 آماده راه‌اندازی ربات تلگرام")
    logger.info(f"{'='*70}\n")


# ==================== نقطه ورود اصلی ====================

if __name__ == "__main__":
    try:
        # تشخیص محیط
        try:
            import IPython
            # محیط Jupyter/Colab
            import nest_asyncio
            nest_asyncio.apply()

            logger.info("🌟 محیط: Jupyter/Colab")

            # اجرای تست
            import asyncio
            loop = asyncio.get_event_loop()
            loop.run_until_complete(main())

            # راه‌اندازی ربات
            logger.info("\n" + "="*70)
            logger.info("🤖 راه‌اندازی ربات تلگرام...")
            logger.info("="*70)

            bot = TelegramBot(TELEGRAM_BOT_TOKEN, TELEGRAM_CHANNEL_ID)

            # ⚠️ تغییر اینجا - استفاده از run_polling بدون بستن loop
            try:
                bot.app = Application.builder().token(bot.token).build()
                bot.app.add_handler(CommandHandler("start", bot.start_command))
                bot.app.add_handler(CommandHandler("help", bot.help_command))
                bot.app.add_handler(CommandHandler("analyze", bot.analyze_command))
                bot.app.add_handler(CommandHandler("stats", bot.stats_command))  # ⬅️ اضافه شد

                logger.info("✅ ربات آماده است")
                logger.info("📱 برای شروع دستور /start را در تلگرام بزنید")

                # اجرای بدون بستن loop
                bot.app.run_polling(allowed_updates=Update.ALL_TYPES, close_loop=False)

            except KeyboardInterrupt:
                logger.info("\n⏹️ ربات توسط کاربر متوقف شد")

        except ImportError:
            # محیط Python معمولی
            logger.info("🖥️ محیط: Python Standard")

            import asyncio
            asyncio.run(main())

            bot = TelegramBot(TELEGRAM_BOT_TOKEN, TELEGRAM_CHANNEL_ID)
            bot.run()

    except Exception as e:
        logger.error(f"\n❌ خطای کلی: {e}")
        logger.error(traceback.format_exc())

    finally:
        logger.info("\n👋 خداحافظ!")

📦 نصب کتابخانه‌های اصلی...
🎨 نصب Kaleido...
✅ Kaleido نصب شد
🔧 نصب تلگرام...
✅ نصب کامل شد

✅ تلگرام بارگذاری شد
Mounted at /content/drive
🌟 محیط Google Colab
📂 BASE_DIR: /content/drive/MyDrive/smart_money_whale_hunter



ERROR:__main__:❌ خطای صرافی kucoin: kucoin does not have market symbol CTK/USDT
ERROR:__main__:❌ خطای صرافی kucoin: kucoin does not have market symbol CTK/USDT
ERROR:__main__:❌ خطای صرافی gate: gate does not have market symbol XPIB/USDT
ERROR:__main__:❌ خطای صرافی kucoin: kucoin does not have market symbol XPIB/USDT
ERROR:__main__:❌ خطای صرافی bybit: bybit does not have market symbol XPIB/USDT
ERROR:__main__:❌ خطای صرافی gate: gate does not have market symbol XPIB/USDT
ERROR:__main__:❌ خطای صرافی kucoin: kucoin does not have market symbol XPIB/USDT
ERROR:__main__:❌ دریافت XPIB/USDT/1h ناموفق بود بعد از 5 تلاش
ERROR:__main__:❌ خطای صرافی kucoin: kucoin does not have market symbol 1COIN/USDT
ERROR:__main__:❌ خطای صرافی kucoin: kucoin does not have market symbol 1COIN/USDT
ERROR:__main__:❌ خطای صرافی gate: gate does not have market symbol 1COIN/USDT
ERROR:__main__:❌ خطای صرافی kucoin: kucoin does not have market symbol 1COIN/USDT
ERROR:__main__:❌ خطای صرافی kucoin: kucoin does not have ma